In [70]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sentence_transformers import util
import numpy as np
import heapq

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

df = pd.read_csv('test.csv', header = 0, sep = ",")
answers = df.iloc[:,1]
student_id = df.iloc[:,0]
sentences = []
for answer in answers:
    sentences.append(answer)
bert_embeddings = model.encode(answers)
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=2)
clustering_model.fit(bert_embeddings)
bert_label = clustering_model.labels_

df_clusters = pd.DataFrame({
    "id": student_id,
    "text": answers,
    "agg_bert_row": bert_label,
})


计算每一类类中心的向量

In [41]:
clusters = {}
groups = {}
for i in range(0,len(student_id)):
    i_cluster = bert_label[i]
    if i_cluster not in clusters:
        groups[i_cluster] = 1
        clusters[i_cluster] = bert_embeddings[i]
    else:
        groups[i_cluster] +=1
        clusters[i_cluster] = clusters[i_cluster] + bert_embeddings[i]
# center of the cluster       
for key in clusters:
    clusters[key] = clusters[key]/groups[key]

找到每一个组的离群点（是否加入老师的input？）

In [67]:
distant_point = {}
distant_point_id = {}
for i in range(0, len(student_id)):
    i_cluster = bert_label[i]
    cos_sim = util.cos_sim(bert_embeddings[i], clusters[i_cluster])
    if i_cluster not in distant_point:
        distant_point[i_cluster] = cos_sim
        distant_point_id[i_cluster] = i
    else:
        if(cos_sim < distant_point[i_cluster]):
            distant_point[i_cluster] = cos_sim
            distant_point_id[i_cluster] = i

print(distant_point_id)

tensor([[0.8526]])
tensor([[0.3395]])
tensor([[0.9934]])
tensor([[0.9910]])
tensor([[0.8349]])
tensor([[0.9926]])
tensor([[0.8355]])
tensor([[0.7568]])
tensor([[0.6483]])
tensor([[0.6996]])
tensor([[0.9452]])
tensor([[0.9112]])
tensor([[0.8370]])
tensor([[0.8483]])
tensor([[0.8533]])
tensor([[0.7256]])
tensor([[0.7659]])
tensor([[0.6918]])
tensor([[0.6790]])
tensor([[0.4895]])
tensor([[0.6839]])
tensor([[0.6413]])
tensor([[0.6538]])
tensor([[0.9959]])
tensor([[0.9812]])
{1: 77, 3: 19, 0: 4, 2: 28, 9: 16, 5: 61, 7: 107, 6: 13, 8: 67, 4: 74}


老师可以在这里选择调整距离

In [93]:
influenced_points = {}
for key in distant_point_id:
    id = distant_point_id[key]
    if(distant_point[key]<0.8):
        key_vector = bert_embeddings[id]
        key_similarity_list = []
        for i in range(0,len(student_id)):
            cos_sim = util.cos_sim(bert_embeddings[id], bert_embeddings[i])
            key_similarity_list.append(cos_sim)
        influenced_points[key] = key_similarity_list

most_similar = {}
for key in influenced_points:
    max_index = []
    max_number = heapq.nlargest(6,influenced_points[key]) 
    for t in max_number:
        index = influenced_points[key].index(t)
        max_index.append(index)
        influenced_points[key][index] = 0
    most_similar[key] = max_index
print(most_similar)
most_similar_answer = {}
for key in most_similar:
    similar_answers = []
    for i in most_similar[key]:
        similar_answer = answers[i]
        similar_answers.append(similar_answer)
    most_similar_answer[key] = similar_answers
print(most_similar_answer)

{1: [77, 66, 0, 86, 68, 95], 3: [19, 70, 1, 43, 91, 17], 0: [4, 2, 16, 21, 38, 98], 5: [61, 8, 48, 11, 23, 29], 8: [67, 14, 24, 65, 43, 56], 4: [74, 20, 36, 22, 99, 2]}
{1: ["It's important to engage in brainstorming", 'Brainstorming', 'Radical ideas ', 'One rule for ideation in human-centered design is to defer judgement while you are brainstorming ideas to be able to have an open mind for any ideas that may arise and to not close off any potential solutions.', 'build off of other ideas', 'Building on others ideas'], 3: ['Defer judgement on ideas, DONT judge them too quickly ', 'one rule of ideation is to defer judgement.', 'No judgement', 'Defer judgement means coming up with ideas, not hating on any of them yet.', "Don't judge ideas to quickly ", "Don't judge other people's ideas"], 0: ['Yes and', 'speed dating', 'include all wild ideas too', 'Go for quantity ', 'Go for quantity', 'Go for quantity'], 5: ['be visual; this is to say that human centered design should focus primarily on